## 🧠 📈 EEG Hand Movement and User Prediction

Given *EEG data from different users performing different hand movements*, let's try to predict the **hand movement** and the user of a given reading.

We will use a Tensorflow neural network to make our predictions.

Data source: https://www.kaggle.com/datasets/fabriciotorquato/eeg-data-from-hands-movement

### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
# pd.set_option('display.max_columns', None)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf

2025-05-21 21:29:46.952763: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
dfs = [pd.read_csv('Dataset/user_' + user + '.csv') for user in ['a', 'b', 'c', 'd']]
dfs

[      Class  AF3 delta std  AF3 delta m  AF3 theta std  AF3 theta m  \
 0       1.0    3569.164550  2063.892754       1.673726     4.444736   
 1       1.0    3568.423670  2063.099248       1.897790     3.728823   
 2       1.0    3568.157929  2062.445859       2.798014     2.574504   
 3       1.0    3567.710021  2062.112673       2.181775     3.610507   
 4       1.0    3565.546124  2063.128867       1.685161     3.384311   
 ...     ...            ...          ...            ...          ...   
 2875    0.0    3572.820630  2064.823341       0.676923     1.620879   
 2876    0.0    3573.915291  2065.845542       0.860279     1.504333   
 2877    0.0    3574.326202  2066.031013       1.416731     2.581577   
 2878    0.0    3573.735788  2065.945333       1.119943     2.769816   
 2879    0.0    3573.982930  2065.960401       1.011062     2.696217   
 
       AF3 alpha std  AF3 alpha m  AF3 beta std  AF3 beta m  F7 delta std  ...  \
 0          0.526209     3.002088      1.425022    3

In [18]:
dfs[0]

,Class,AF3 delta std,AF3 delta m,AF3 theta std,AF3 theta m,AF3 alpha std,AF3 alpha m,AF3 beta std,AF3 beta m,F7 delta std,...,F8 beta std,F8 beta m,AF4 delta std,AF4 delta m,AF4 theta std,AF4 theta m,AF4 alpha std,AF4 alpha m,AF4 beta std,AF4 beta m
0,1.0,3569.164550,2063.892754,1.673726,4.444736,0.526209,3.002088,1.425022,3.302739,3563.803888,...,45.468326,72.508750,3701.186330,2182.676835,18.192418,41.349662,16.004756,42.046467,46.280843,73.565719
1,1.0,3568.423670,2063.099248,1.897790,3.728823,1.304186,1.854353,1.366575,2.546458,3563.560922,...,36.551948,66.931186,3725.210509,2180.197439,8.820788,38.012788,19.601233,29.431054,38.559351,67.470041
2,1.0,3568.157929,2062.445859,2.798014,2.574504,1.120537,1.958819,0.982433,2.258622,3563.279981,...,40.754308,66.816547,3724.417296,2176.823208,18.159202,23.612639,14.378291,19.555084,43.210004,67.781924
3,1.0,3567.710021,2062.112673,2.181775,3.610507,0.629608,2.155876,0.856275,2.233711,3562.787801,...,38.074628,63.915386,3725.822160,2177.089059,19.737616,29.484396,15.793034,25.713513,39.250246,65.031031
4,1.0,3565.546124,2063.128867,1.685161,3.384311,0.677526,1.795798,0.927924,1.909810,3562.655091,...,35.357384,64.534645,3723.053978,2167.798335,8.429414,26.374975,14.920736,35.675266,33.901687,66.956313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875,0.0,3572.820630,2064.823341,0.676923,1.620879,0.965369,2.392263,2.582266,3.185588,3565.166724,...,26.247669,13.320391,3607.467804,2089.263161,1.605773,4.247634,1.649762,2.938868,31.019482,16.648758
2876,0.0,3573.915291,2065.845542,0.860279,1.504333,0.539180,1.331165,2.533695,2.870270,3565.734750,...,26.120266,14.809811,3613.165565,2089.298802,0.643899,6.903858,1.287563,3.942647,31.546882,17.429113
2877,0.0,3574.326202,2066.031013,1.416731,2.581577,0.715012,1.307975,2.261013,2.873613,3567.582880,...,26.513829,16.084048,3615.755534,2094.482958,1.851314,5.683580,2.411642,6.355117,32.238063,19.023897
2878,0.0,3573.735788,2065.945333,1.119943,2.769816,0.455408,1.541430,1.782814,3.067384,3567.857997,...,20.049092,21.248860,3620.821019,2097.450420,5.035767,12.027902,6.441267,14.663681,24.109509,26.468254


In [19]:
pd.Series(0, index=dfs[0].index)

0       0
1       0
2       0
3       0
4       0
       ..
2875    0
2876    0
2877    0
2878    0
2879    0
Length: 2880, dtype: int64

In [20]:
for i in range(len(dfs)):
    dfs[i]['User'] = pd.Series(i, index=dfs[i].index)

In [21]:
data = pd.concat(dfs, axis=0).sample(frac=1.0, random_state=123).reset_index(drop=True)
data

,Class,AF3 delta std,AF3 delta m,AF3 theta std,AF3 theta m,AF3 alpha std,AF3 alpha m,AF3 beta std,AF3 beta m,F7 delta std,...,F8 beta m,AF4 delta std,AF4 delta m,AF4 theta std,AF4 theta m,AF4 alpha std,AF4 alpha m,AF4 beta std,AF4 beta m,User
0,0.0,3572.252440,2065.056469,0.851824,2.047953,0.651871,2.522036,2.109733,3.347705,3564.779879,...,35.151586,3628.426885,2129.789645,5.353671,17.885132,7.672209,29.960618,43.216980,43.932669,0
1,1.0,3574.116024,2065.528155,1.932513,3.321636,1.138012,2.349805,2.256212,3.945981,3563.399422,...,40.800889,3680.341349,2144.200503,10.819521,36.995982,12.812193,24.146774,23.747501,49.072017,0
2,0.0,3554.487593,2056.215665,0.935015,3.793783,0.736168,2.680542,3.381325,4.678876,3568.839949,...,18.176841,3538.347368,2081.315814,5.486555,13.204753,0.664075,6.633072,1.434277,4.132446,2
3,0.0,3570.668125,2063.974908,1.875394,3.028541,0.910000,3.018672,1.163312,2.883009,3565.403408,...,55.547547,3604.601528,2122.493834,15.611283,16.452483,22.462175,46.703612,32.213578,70.892466,0
4,0.0,3559.747108,2057.401763,1.053691,1.530594,1.593121,2.789907,2.668865,4.326693,3573.651774,...,7.208052,3513.244789,2030.461207,1.455450,3.030659,0.482971,3.200647,0.895170,2.099638,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11515,2.0,3570.748191,2063.201099,0.727594,1.404708,0.995674,2.569811,0.991322,1.987807,3562.594707,...,1.910036,3567.687654,2062.791757,1.748982,2.097780,1.242056,1.682180,1.277379,1.997654,3
11516,1.0,3566.600695,2062.436502,1.491797,3.013466,1.338308,2.769207,4.268401,3.877338,3576.187723,...,5.217772,3548.277991,2055.537892,1.787661,3.366967,0.810903,1.739812,2.202195,3.294145,2
11517,0.0,3574.186933,2064.477869,0.710708,1.534898,0.749206,2.038258,2.208018,2.157076,3566.398415,...,2.200203,3572.290285,2063.776063,1.268569,2.547969,0.887808,1.651011,1.533523,1.957866,1
11518,0.0,3574.343116,2065.996679,0.606401,2.835015,0.617079,1.209722,3.357912,3.098423,3566.521533,...,15.413625,3608.394990,2091.509508,1.479338,3.824176,3.281520,4.238889,33.441194,19.158094,0


In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11520 entries, 0 to 11519
Columns: 114 entries, Class to User
dtypes: float64(113), int64(1)
memory usage: 10.0 MB


In [23]:
data.select_dtypes('object').columns

Index([], dtype='object')

#### Helper Functions

In [24]:
def onehot_encode(df, column):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=column, dtype=int)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

### Preprocessing

In [25]:
df = data.copy()

#### Predicting Hand Movement Class

In [26]:
# One-hot encode Class column 
df = onehot_encode(df, column='User')

In [27]:
# Split df into X and y
y = df['Class'].copy()
X = df.drop('Class', axis=1).copy()

In [28]:
X

,AF3 delta std,AF3 delta m,AF3 theta std,AF3 theta m,AF3 alpha std,AF3 alpha m,AF3 beta std,AF3 beta m,F7 delta std,F7 delta m,...,AF4 theta std,AF4 theta m,AF4 alpha std,AF4 alpha m,AF4 beta std,AF4 beta m,User_0,User_1,User_2,User_3
0,3572.252440,2065.056469,0.851824,2.047953,0.651871,2.522036,2.109733,3.347705,3564.779879,2061.195445,...,5.353671,17.885132,7.672209,29.960618,43.216980,43.932669,1,0,0,0
1,3574.116024,2065.528155,1.932513,3.321636,1.138012,2.349805,2.256212,3.945981,3563.399422,2062.290606,...,10.819521,36.995982,12.812193,24.146774,23.747501,49.072017,1,0,0,0
2,3554.487593,2056.215665,0.935015,3.793783,0.736168,2.680542,3.381325,4.678876,3568.839949,2061.818772,...,5.486555,13.204753,0.664075,6.633072,1.434277,4.132446,0,0,1,0
3,3570.668125,2063.974908,1.875394,3.028541,0.910000,3.018672,1.163312,2.883009,3565.403408,2060.271106,...,15.611283,16.452483,22.462175,46.703612,32.213578,70.892466,1,0,0,0
4,3559.747108,2057.401763,1.053691,1.530594,1.593121,2.789907,2.668865,4.326693,3573.651774,2065.274034,...,1.455450,3.030659,0.482971,3.200647,0.895170,2.099638,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11515,3570.748191,2063.201099,0.727594,1.404708,0.995674,2.569811,0.991322,1.987807,3562.594707,2059.455684,...,1.748982,2.097780,1.242056,1.682180,1.277379,1.997654,0,0,0,1
11516,3566.600695,2062.436502,1.491797,3.013466,1.338308,2.769207,4.268401,3.877338,3576.187723,2065.982437,...,1.787661,3.366967,0.810903,1.739812,2.202195,3.294145,0,0,1,0
11517,3574.186933,2064.477869,0.710708,1.534898,0.749206,2.038258,2.208018,2.157076,3566.398415,2061.123322,...,1.268569,2.547969,0.887808,1.651011,1.533523,1.957866,0,1,0,0
11518,3574.343116,2065.996679,0.606401,2.835015,0.617079,1.209722,3.357912,3.098423,3566.521533,2060.758758,...,1.479338,3.824176,3.281520,4.238889,33.441194,19.158094,1,0,0,0


In [29]:
y

0        0.0
1        1.0
2        0.0
3        0.0
4        0.0
        ... 
11515    2.0
11516    1.0
11517    0.0
11518    0.0
11519    1.0
Name: Class, Length: 11520, dtype: float64

In [30]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=123)

In [31]:
# Scale X with a standard scaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)

In [32]:
X_train

,AF3 delta std,AF3 delta m,AF3 theta std,AF3 theta m,AF3 alpha std,AF3 alpha m,AF3 beta std,AF3 beta m,F7 delta std,F7 delta m,...,AF4 theta std,AF4 theta m,AF4 alpha std,AF4 alpha m,AF4 beta std,AF4 beta m,User_0,User_1,User_2,User_3
8701,-0.592492,-0.628348,-0.961159,0.055905,0.463966,0.243742,-1.048130,-0.492935,-1.039009,-0.724031,...,-0.436216,-0.408949,-0.407029,-0.490410,-0.536367,-0.539566,-0.574343,-0.578735,-0.575870,1.722792
4335,-1.215542,-1.060438,-0.687275,0.911497,0.266495,-0.045726,-0.996263,-1.073110,-1.595935,-1.789505,...,2.577270,2.173543,3.159511,3.022871,0.657025,2.073117,1.741120,-0.578735,-0.575870,-0.580453
11203,0.217034,0.061461,-0.302453,-0.276674,-0.609120,-0.737363,-1.144599,-1.598413,-1.092270,-1.223951,...,-0.400908,-0.385079,-0.267008,-0.348359,-0.541364,-0.569416,-0.574343,-0.578735,-0.575870,1.722792
11448,2.636827,3.089887,0.715433,1.688836,1.115520,0.633049,-0.753087,-0.640998,0.643631,0.970393,...,0.141047,-0.008948,-0.427868,-0.301202,-0.505539,-0.522675,-0.574343,-0.578735,-0.575870,1.722792
374,-0.012351,0.039718,-0.098249,-0.213172,0.948769,-0.058219,-0.254097,-0.487547,0.332959,0.028684,...,-0.565201,-0.479292,-0.372068,-0.497185,-0.523806,-0.525149,-0.574343,1.727908,-0.575870,-0.580453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9785,0.618517,1.150023,-0.957440,-0.735247,0.634060,1.144167,1.837545,2.507360,1.076999,1.533723,...,-0.278668,0.153400,-0.380571,-0.152910,-0.500346,-0.453114,-0.574343,-0.578735,1.736502,-0.580453
7763,-0.157839,-0.326705,-0.099050,0.460753,-0.022372,-0.179817,1.098608,0.432161,-0.522874,-0.840956,...,2.779410,1.612038,1.694883,4.498588,3.159293,2.567176,1.741120,-0.578735,-0.575870,-0.580453
5218,0.166508,0.010035,0.018053,-0.268545,-1.090114,0.455020,0.757639,1.987062,0.999285,0.692423,...,-0.430248,-0.416324,-0.096056,-0.372181,-0.508712,-0.493789,-0.574343,-0.578735,1.736502,-0.580453
1346,0.202509,-0.017317,-0.267921,-0.589426,0.478543,-0.449163,-0.196672,-0.763750,-0.557585,-0.719922,...,-0.238893,-0.510057,-0.441859,-0.499372,-0.511246,-0.538075,-0.574343,1.727908,-0.575870,-0.580453


In [34]:
X_train.mean()

AF3 delta std   -1.766531e-14
AF3 delta m      4.311372e-14
AF3 theta std    9.429254e-17
AF3 theta m      1.136798e-16
AF3 alpha std   -1.982787e-16
                     ...     
AF4 beta m       4.758689e-17
User_0           2.555592e-17
User_1          -1.172047e-16
User_2          -1.185266e-16
User_3          -4.009636e-17
Length: 116, dtype: float64

In [35]:
X_train.var()

AF3 delta std    1.000124
AF3 delta m      1.000124
AF3 theta std    1.000124
AF3 theta m      1.000124
AF3 alpha std    1.000124
                   ...   
AF4 beta m       1.000124
User_0           1.000124
User_1           1.000124
User_2           1.000124
User_3           1.000124
Length: 116, dtype: float64

In [36]:
y_train

8701     1.0
4335     1.0
11203    1.0
11448    0.0
374      2.0
        ... 
9785     1.0
7763     2.0
5218     1.0
1346     1.0
3582     2.0
Name: Class, Length: 8063, dtype: float64

In [37]:
y_train.value_counts()

Class
2.0    2716
0.0    2698
1.0    2649
Name: count, dtype: int64

In [33]:
X_train.shape, X_test.shape

((8063, 116), (3457, 116))

In [38]:
def build_model(num_classes=3):
    inputs = tf.keras.Input(shape=(X_train.shape[1],))
    x = tf.keras.layers.Dense(128, activation='relu')(inputs)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [39]:
class_model = build_model(num_classes=3)

class_history = class_model.fit(
    X_train,
    y_train,
    validation_split = 0.2,
    batch_size = 32,
    epochs = 50,
    callbacks = [tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience = 3,
        restore_best_weights = True
    )]
)

2025-05-21 22:32:39.658392: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/50
202/202 [==============================] - 2s 7ms/step - loss: 1.0657 - accuracy: 0.4319 - val_loss: 1.0262 - val_accuracy: 0.4582
Epoch 2/50
202/202 [==============================] - 1s 6ms/step - loss: 0.9788 - accuracy: 0.5078 - val_loss: 0.9816 - val_accuracy: 0.4978
Epoch 3/50
202/202 [==============================] - 1s 5ms/step - loss: 0.9182 - accuracy: 0.5581 - val_loss: 0.9586 - val_accuracy: 0.5257
Epoch 4/50
202/202 [==============================] - 1s 4ms/step - loss: 0.8697 - accuracy: 0.5896 - val_loss: 0.9072 - val_accuracy: 0.5642
Epoch 5/50
202/202 [==============================] - 1s 7ms/step - loss: 0.8299 - accuracy: 0.6175 - val_loss: 0.9076 - val_accuracy: 0.5766
Epoch 6/50
202/202 [==============================] - 1s 5ms/step - loss: 0.7879 - accuracy: 0.6419 - val_loss: 0.8684 - val_accuracy: 0.6001
Epoch 7/50
202/202 [==============================] - 1s 5ms/step - loss: 0.7392 - accuracy: 0.6733 - val_loss: 0.8797 - val_accuracy: 0.5945
Epoch 

In [42]:
class_acc = class_model.evaluate(X_test, y_test, verbose=0)[1]
print("Test Accuracy (Class Model): {:.2f}%".format(class_acc*100))

Test Accuracy (Class Model): 63.09%


#### Predicting User

In [43]:
df = data.copy()

In [44]:
# One-hot encode User column 
df = onehot_encode(df, column='Class')

In [45]:
# Split df into X and y
y = df['User'].copy()
X = df.drop('User', axis=1).copy()

In [46]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=123)

In [49]:
# Scale X with a standard scaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)

In [51]:
X_train

,AF3 delta std,AF3 delta m,AF3 theta std,AF3 theta m,AF3 alpha std,AF3 alpha m,AF3 beta std,AF3 beta m,F7 delta std,F7 delta m,...,AF4 delta m,AF4 theta std,AF4 theta m,AF4 alpha std,AF4 alpha m,AF4 beta std,AF4 beta m,Class_0.0,Class_1.0,Class_2.0
8701,-0.592492,-0.628348,-0.961159,0.055905,0.463966,0.243742,-1.048130,-0.492935,-1.039009,-0.724031,...,-0.535664,-0.436216,-0.408949,-0.407029,-0.490410,-0.536367,-0.539566,-0.709147,1.429612,-0.712705
4335,-1.215542,-1.060438,-0.687275,0.911497,0.266495,-0.045726,-0.996263,-1.073110,-1.595935,-1.789505,...,3.858153,2.577270,2.173543,3.159511,3.022871,0.657025,2.073117,-0.709147,1.429612,-0.712705
11203,0.217034,0.061461,-0.302453,-0.276674,-0.609120,-0.737363,-1.144599,-1.598413,-1.092270,-1.223951,...,-0.537446,-0.400908,-0.385079,-0.267008,-0.348359,-0.541364,-0.569416,-0.709147,1.429612,-0.712705
11448,2.636827,3.089887,0.715433,1.688836,1.115520,0.633049,-0.753087,-0.640998,0.643631,0.970393,...,-0.382336,0.141047,-0.008948,-0.427868,-0.301202,-0.505539,-0.522675,1.410145,-0.699491,-0.712705
374,-0.012351,0.039718,-0.098249,-0.213172,0.948769,-0.058219,-0.254097,-0.487547,0.332959,0.028684,...,-0.465464,-0.565201,-0.479292,-0.372068,-0.497185,-0.523806,-0.525149,-0.709147,-0.699491,1.403105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9785,0.618517,1.150023,-0.957440,-0.735247,0.634060,1.144167,1.837545,2.507360,1.076999,1.533723,...,1.371221,-0.278668,0.153400,-0.380571,-0.152910,-0.500346,-0.453114,-0.709147,1.429612,-0.712705
7763,-0.157839,-0.326705,-0.099050,0.460753,-0.022372,-0.179817,1.098608,0.432161,-0.522874,-0.840956,...,1.627338,2.779410,1.612038,1.694883,4.498588,3.159293,2.567176,-0.709147,-0.699491,1.403105
5218,0.166508,0.010035,0.018053,-0.268545,-1.090114,0.455020,0.757639,1.987062,0.999285,0.692423,...,-0.167387,-0.430248,-0.416324,-0.096056,-0.372181,-0.508712,-0.493789,-0.709147,1.429612,-0.712705
1346,0.202509,-0.017317,-0.267921,-0.589426,0.478543,-0.449163,-0.196672,-0.763750,-0.557585,-0.719922,...,-0.463766,-0.238893,-0.510057,-0.441859,-0.499372,-0.511246,-0.538075,-0.709147,1.429612,-0.712705


In [50]:
y_train.value_counts()

User
3    2032
1    2023
2    2008
0    2000
Name: count, dtype: int64

In [52]:
user_model = build_model(num_classes=4)

user_history = user_model.fit(
    X_train,
    y_train,
    validation_split = 0.2,
    batch_size = 32,
    epochs = 50,
    callbacks = [tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience = 3,
        restore_best_weights = True
    )]
)

Epoch 1/50
202/202 [==============================] - 2s 7ms/step - loss: 0.1159 - accuracy: 0.9626 - val_loss: 0.0238 - val_accuracy: 0.9926
Epoch 2/50
202/202 [==============================] - 1s 6ms/step - loss: 0.0143 - accuracy: 0.9960 - val_loss: 0.0131 - val_accuracy: 0.9963
Epoch 3/50
202/202 [==============================] - 1s 5ms/step - loss: 0.0067 - accuracy: 0.9988 - val_loss: 0.0065 - val_accuracy: 0.9981
Epoch 4/50
202/202 [==============================] - 1s 5ms/step - loss: 0.0048 - accuracy: 0.9984 - val_loss: 0.0048 - val_accuracy: 0.9981
Epoch 5/50
202/202 [==============================] - 1s 3ms/step - loss: 0.0020 - accuracy: 0.9998 - val_loss: 0.0060 - val_accuracy: 0.9975
Epoch 6/50
202/202 [==============================] - 1s 3ms/step - loss: 9.1140e-04 - accuracy: 1.0000 - val_loss: 0.0032 - val_accuracy: 0.9988
Epoch 7/50
202/202 [==============================] - 1s 3ms/step - loss: 6.1568e-04 - accuracy: 1.0000 - val_loss: 0.0046 - val_accuracy: 0.998

In [53]:
user_acc = user_model.evaluate(X_test, y_test, verbose=0)[1]
print("Test Accuracy (User Model): {:.2f}%".format(user_acc*100))

Test Accuracy (User Model): 99.94%
